In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import datasets
from tensorflow.keras import callbacks
from tensorflow.keras.datasets import mnist
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications import EfficientNetB7
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.applications import EfficientNetB0

import tensorflow_datasets as tfds

import matplotlib.pyplot as plt

import os

import tempfile
from os import path
%load_ext tensorboard

def normalize(image, label):
    return tf.cast(image, tf.float32) / 255., label

# Global settings

In [2]:
# todo change this to 100+

NUM_EPOCHS = 300

### Train and save model 

In [3]:
def train_model(model, ds_train, ds_validation, model_name, batch_size=64):
#    if path.exists(model_path):
#        print("Model is already trained and saved here: " + model_path)
#        return
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
    
    
    save = callbacks.ModelCheckpoint(
        os.path.join('tmp', model_name + "_e_{epoch:02d}.h5"),
        monitor='loss',
        verbose=1,
        save_best_only=False,
        save_weights_only=False,
        mode='auto')

    early = callbacks.EarlyStopping(monitor='val_loss',
                                    min_delta=0,
                                    patience=30,
                                    verbose=1,
                                    mode='auto')

    hist = model.fit(ds_train, 
                     epochs=NUM_EPOCHS, 
                     validation_data=ds_validation,
                     callbacks=[save]
                    )
    
    # save model
    model.save(model_name + ".h5")
    print('Saved to: ' + model_name + ".h5")

# Image classification models

In [4]:
def convert_ds_to_tensors(ds):
    """returns tuple of train_X, train_y"""
    a = ds.map(lambda a, b: a)
    tf_list = []
    for i in a:
        tf_list.append(i)
    train_X = tf.stack(tf_list, axis=0)
    
    b = ds.map(lambda a, b: b)
    tf_list = []
    for i in b:
        tf_list.append([i.numpy()])
    train_y = np.array(tf_list, dtype=np.uint8)
    return train_X, train_y

## Beans dataset

https://www.tensorflow.org/datasets/catalog/beans

In [5]:
def preprocess_beans_train(image, label):
    image = tf.image.random_flip_left_right(image)    
    return image, label

def preprocess_beans_test_and_val(image, label):
    return image, label

### wrap model for beans

In [6]:
def wrap_model_for_beans(base_model, num_classes):
    inputs = base_model.inputs
    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.2)(x)
    outputs = layers.Dense(num_classes)(x)
    model = tf.keras.Model(inputs, outputs)
    return model

In [7]:
NUM_CLASSES = 3
INPUT_SHAPE = (500, 500, 3)
BATCH_SIZE = 32

In [8]:
def load_beans_datasets():
    (ds_train, ds_validation, ds_test), ds_info = tfds.load(
        'beans',
        split=['train', 'validation', 'test'],
        shuffle_files=True,
        as_supervised=True,
        with_info=True,
    )
    
    ds_train = ds_train.map(normalize)
    #ds_train = ds_train.map(preprocess_beans_train)
    ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples).batch(BATCH_SIZE)
    
    ds_validation = ds_validation.map(normalize)
    #ds_validation = ds_validation.map(preprocess_beans_test_and_val)
    ds_validation = ds_validation.batch(BATCH_SIZE)
    
    ds_test = ds_test.map(normalize)
    #ds_test = ds_test.map(preprocess_beans_test_and_val)
    ds_test = ds_test.batch(BATCH_SIZE)
    
    
    return ds_train, ds_validation, ds_test

In [9]:
ds_train, ds_validation, ds_test = load_beans_datasets()


### mobilenetV2

In [10]:
MODEL_NAME = 'MobileNetV2_beans_model'

In [11]:
base_model = MobileNetV2(include_top=False, weights=None, input_shape=INPUT_SHAPE)

model = wrap_model_for_beans(base_model=base_model, num_classes=NUM_CLASSES)

train_model(model=model, ds_train=ds_train, ds_validation=ds_validation, model_name=MODEL_NAME)

Epoch 1/300
33/33 [==============================] - 40s 951ms/step - loss: 1.0113 - sparse_categorical_accuracy: 0.4955 - val_loss: 1.0988 - val_sparse_categorical_accuracy: 0.3383

Epoch 00001: saving model to tmp/MobileNetV2_beans_model_e_01.h5
Epoch 2/300
33/33 [==============================] - 33s 892ms/step - loss: 0.7942 - sparse_categorical_accuracy: 0.6594 - val_loss: 1.0999 - val_sparse_categorical_accuracy: 0.3383

Epoch 00002: saving model to tmp/MobileNetV2_beans_model_e_02.h5
Epoch 3/300
33/33 [==============================] - 33s 911ms/step - loss: 0.7157 - sparse_categorical_accuracy: 0.6887 - val_loss: 1.1011 - val_sparse_categorical_accuracy: 0.3383

Epoch 00003: saving model to tmp/MobileNetV2_beans_model_e_03.h5
Epoch 4/300
33/33 [==============================] - 33s 887ms/step - loss: 0.6170 - sparse_categorical_accuracy: 0.7321 - val_loss: 1.1077 - val_sparse_categorical_accuracy: 0.3383

Epoch 00004: saving model to tmp/MobileNetV2_beans_model_e_04.h5
Epoch 5/

Epoch 34/300
33/33 [==============================] - 33s 904ms/step - loss: 0.0628 - sparse_categorical_accuracy: 0.9837 - val_loss: 2.4276 - val_sparse_categorical_accuracy: 0.3383

Epoch 00034: saving model to tmp/MobileNetV2_beans_model_e_34.h5
Epoch 35/300
33/33 [==============================] - 32s 885ms/step - loss: 0.0437 - sparse_categorical_accuracy: 0.9847 - val_loss: 2.6159 - val_sparse_categorical_accuracy: 0.3383

Epoch 00035: saving model to tmp/MobileNetV2_beans_model_e_35.h5
Epoch 36/300
33/33 [==============================] - 33s 923ms/step - loss: 0.0382 - sparse_categorical_accuracy: 0.9843 - val_loss: 2.6075 - val_sparse_categorical_accuracy: 0.3383

Epoch 00036: saving model to tmp/MobileNetV2_beans_model_e_36.h5
Epoch 37/300
33/33 [==============================] - 33s 904ms/step - loss: 0.0338 - sparse_categorical_accuracy: 0.9860 - val_loss: 2.6848 - val_sparse_categorical_accuracy: 0.3383

Epoch 00037: saving model to tmp/MobileNetV2_beans_model_e_37.h5
Epoc

Epoch 67/300
33/33 [==============================] - 32s 888ms/step - loss: 0.0606 - sparse_categorical_accuracy: 0.9877 - val_loss: 3.0392 - val_sparse_categorical_accuracy: 0.3383

Epoch 00067: saving model to tmp/MobileNetV2_beans_model_e_67.h5
Epoch 68/300
33/33 [==============================] - 33s 899ms/step - loss: 0.0303 - sparse_categorical_accuracy: 0.9948 - val_loss: 2.9968 - val_sparse_categorical_accuracy: 0.3383

Epoch 00068: saving model to tmp/MobileNetV2_beans_model_e_68.h5
Epoch 69/300
33/33 [==============================] - 33s 902ms/step - loss: 0.0090 - sparse_categorical_accuracy: 0.9972 - val_loss: 3.0462 - val_sparse_categorical_accuracy: 0.3383

Epoch 00069: saving model to tmp/MobileNetV2_beans_model_e_69.h5
Epoch 70/300
33/33 [==============================] - 32s 905ms/step - loss: 0.0174 - sparse_categorical_accuracy: 0.9906 - val_loss: 2.8533 - val_sparse_categorical_accuracy: 0.3383

Epoch 00070: saving model to tmp/MobileNetV2_beans_model_e_70.h5
Epoc

Epoch 100/300
33/33 [==============================] - 33s 907ms/step - loss: 0.0067 - sparse_categorical_accuracy: 0.9993 - val_loss: 2.5970 - val_sparse_categorical_accuracy: 0.3308

Epoch 00100: saving model to tmp/MobileNetV2_beans_model_e_100.h5
Epoch 101/300
33/33 [==============================] - 32s 895ms/step - loss: 0.0032 - sparse_categorical_accuracy: 1.0000 - val_loss: 2.5848 - val_sparse_categorical_accuracy: 0.3308

Epoch 00101: saving model to tmp/MobileNetV2_beans_model_e_101.h5
Epoch 102/300
33/33 [==============================] - 32s 889ms/step - loss: 0.0050 - sparse_categorical_accuracy: 0.9983 - val_loss: 2.5315 - val_sparse_categorical_accuracy: 0.3308

Epoch 00102: saving model to tmp/MobileNetV2_beans_model_e_102.h5
Epoch 103/300
33/33 [==============================] - 32s 888ms/step - loss: 0.0039 - sparse_categorical_accuracy: 0.9998 - val_loss: 2.5037 - val_sparse_categorical_accuracy: 0.3308

Epoch 00103: saving model to tmp/MobileNetV2_beans_model_e_103


Epoch 00132: saving model to tmp/MobileNetV2_beans_model_e_132.h5
Epoch 133/300
33/33 [==============================] - 32s 893ms/step - loss: 0.0204 - sparse_categorical_accuracy: 0.9927 - val_loss: 3.4513 - val_sparse_categorical_accuracy: 0.3308

Epoch 00133: saving model to tmp/MobileNetV2_beans_model_e_133.h5
Epoch 134/300
33/33 [==============================] - 32s 890ms/step - loss: 0.0065 - sparse_categorical_accuracy: 0.9967 - val_loss: 3.5356 - val_sparse_categorical_accuracy: 0.3308

Epoch 00134: saving model to tmp/MobileNetV2_beans_model_e_134.h5
Epoch 135/300
33/33 [==============================] - 32s 884ms/step - loss: 0.0053 - sparse_categorical_accuracy: 0.9990 - val_loss: 3.4644 - val_sparse_categorical_accuracy: 0.3308

Epoch 00135: saving model to tmp/MobileNetV2_beans_model_e_135.h5
Epoch 136/300
33/33 [==============================] - 32s 900ms/step - loss: 0.0190 - sparse_categorical_accuracy: 0.9946 - val_loss: 3.3916 - val_sparse_categorical_accuracy: 0.3

33/33 [==============================] - 33s 905ms/step - loss: 0.0021 - sparse_categorical_accuracy: 1.0000 - val_loss: 3.5119 - val_sparse_categorical_accuracy: 0.3308

Epoch 00165: saving model to tmp/MobileNetV2_beans_model_e_165.h5
Epoch 166/300
33/33 [==============================] - 32s 900ms/step - loss: 0.0035 - sparse_categorical_accuracy: 0.9983 - val_loss: 3.5845 - val_sparse_categorical_accuracy: 0.3308

Epoch 00166: saving model to tmp/MobileNetV2_beans_model_e_166.h5
Epoch 167/300
33/33 [==============================] - 32s 899ms/step - loss: 0.0034 - sparse_categorical_accuracy: 0.9998 - val_loss: 3.4601 - val_sparse_categorical_accuracy: 0.3308

Epoch 00167: saving model to tmp/MobileNetV2_beans_model_e_167.h5
Epoch 168/300
33/33 [==============================] - 32s 896ms/step - loss: 0.0292 - sparse_categorical_accuracy: 0.9876 - val_loss: 4.1157 - val_sparse_categorical_accuracy: 0.3308

Epoch 00168: saving model to tmp/MobileNetV2_beans_model_e_168.h5
Epoch 169/


Epoch 00197: saving model to tmp/MobileNetV2_beans_model_e_197.h5
Epoch 198/300
33/33 [==============================] - 33s 913ms/step - loss: 0.0571 - sparse_categorical_accuracy: 0.9776 - val_loss: 3.7284 - val_sparse_categorical_accuracy: 0.7143

Epoch 00198: saving model to tmp/MobileNetV2_beans_model_e_198.h5
Epoch 199/300
33/33 [==============================] - 32s 893ms/step - loss: 0.0574 - sparse_categorical_accuracy: 0.9849 - val_loss: 3.4972 - val_sparse_categorical_accuracy: 0.6617

Epoch 00199: saving model to tmp/MobileNetV2_beans_model_e_199.h5
Epoch 200/300
33/33 [==============================] - 32s 897ms/step - loss: 0.0393 - sparse_categorical_accuracy: 0.9838 - val_loss: 2.4087 - val_sparse_categorical_accuracy: 0.7068

Epoch 00200: saving model to tmp/MobileNetV2_beans_model_e_200.h5
Epoch 201/300
33/33 [==============================] - 32s 900ms/step - loss: 0.0151 - sparse_categorical_accuracy: 0.9942 - val_loss: 3.1148 - val_sparse_categorical_accuracy: 0.6

33/33 [==============================] - 32s 894ms/step - loss: 0.0080 - sparse_categorical_accuracy: 0.9984 - val_loss: 1.7926 - val_sparse_categorical_accuracy: 0.7594

Epoch 00230: saving model to tmp/MobileNetV2_beans_model_e_230.h5
Epoch 231/300
33/33 [==============================] - 32s 908ms/step - loss: 0.0057 - sparse_categorical_accuracy: 0.9977 - val_loss: 5.7187 - val_sparse_categorical_accuracy: 0.6165

Epoch 00231: saving model to tmp/MobileNetV2_beans_model_e_231.h5
Epoch 232/300
33/33 [==============================] - 33s 909ms/step - loss: 0.0177 - sparse_categorical_accuracy: 0.9965 - val_loss: 2.1374 - val_sparse_categorical_accuracy: 0.7368

Epoch 00232: saving model to tmp/MobileNetV2_beans_model_e_232.h5
Epoch 233/300
33/33 [==============================] - 33s 917ms/step - loss: 0.0030 - sparse_categorical_accuracy: 0.9993 - val_loss: 1.9002 - val_sparse_categorical_accuracy: 0.7519

Epoch 00233: saving model to tmp/MobileNetV2_beans_model_e_233.h5
Epoch 234/


Epoch 00262: saving model to tmp/MobileNetV2_beans_model_e_262.h5
Epoch 263/300
33/33 [==============================] - 32s 887ms/step - loss: 0.0019 - sparse_categorical_accuracy: 0.9991 - val_loss: 1.9124 - val_sparse_categorical_accuracy: 0.7820

Epoch 00263: saving model to tmp/MobileNetV2_beans_model_e_263.h5
Epoch 264/300
33/33 [==============================] - 32s 892ms/step - loss: 0.0014 - sparse_categorical_accuracy: 1.0000 - val_loss: 1.9989 - val_sparse_categorical_accuracy: 0.7895

Epoch 00264: saving model to tmp/MobileNetV2_beans_model_e_264.h5
Epoch 265/300
33/33 [==============================] - 33s 907ms/step - loss: 0.0014 - sparse_categorical_accuracy: 0.9996 - val_loss: 2.2001 - val_sparse_categorical_accuracy: 0.7594

Epoch 00265: saving model to tmp/MobileNetV2_beans_model_e_265.h5
Epoch 266/300
33/33 [==============================] - 33s 912ms/step - loss: 0.0015 - sparse_categorical_accuracy: 1.0000 - val_loss: 2.5320 - val_sparse_categorical_accuracy: 0.7

33/33 [==============================] - 32s 890ms/step - loss: 0.0374 - sparse_categorical_accuracy: 0.9846 - val_loss: 12.9825 - val_sparse_categorical_accuracy: 0.5188

Epoch 00295: saving model to tmp/MobileNetV2_beans_model_e_295.h5
Epoch 296/300
33/33 [==============================] - 32s 890ms/step - loss: 0.0131 - sparse_categorical_accuracy: 0.9948 - val_loss: 9.8595 - val_sparse_categorical_accuracy: 0.5789

Epoch 00296: saving model to tmp/MobileNetV2_beans_model_e_296.h5
Epoch 297/300
33/33 [==============================] - 33s 916ms/step - loss: 0.0338 - sparse_categorical_accuracy: 0.9864 - val_loss: 10.5916 - val_sparse_categorical_accuracy: 0.5263

Epoch 00297: saving model to tmp/MobileNetV2_beans_model_e_297.h5
Epoch 298/300
33/33 [==============================] - 32s 888ms/step - loss: 0.0454 - sparse_categorical_accuracy: 0.9881 - val_loss: 5.8728 - val_sparse_categorical_accuracy: 0.6090

Epoch 00298: saving model to tmp/MobileNetV2_beans_model_e_298.h5
Epoch 29

In [12]:
model.evaluate(ds_test)

4/4 [==============================] - 1s 140ms/step - loss: 12.9655 - sparse_categorical_accuracy: 0.5312


[12.965460777282715, 0.53125]

### EfficientNets - B0, B4

#### EfficientNetB0

In [13]:
MODEL_PATH = "EfficentNetB0_beans_model"

In [14]:
base_model = EfficientNetB0(include_top=False, weights=None, input_shape=INPUT_SHAPE)

model = wrap_model_for_beans(base_model=base_model, num_classes=NUM_CLASSES)

train_model(model=model, ds_train=ds_train, ds_validation=ds_validation, model_name=MODEL_PATH)

Epoch 1/300
33/33 [==============================] - 51s 1s/step - loss: 1.2395 - sparse_categorical_accuracy: 0.3663 - val_loss: 1.1039 - val_sparse_categorical_accuracy: 0.3383

Epoch 00001: saving model to tmp/EfficentNetB0_beans_model_e_01.h5
Epoch 2/300
33/33 [==============================] - 45s 1s/step - loss: 0.9288 - sparse_categorical_accuracy: 0.5635 - val_loss: 1.2294 - val_sparse_categorical_accuracy: 0.3308

Epoch 00002: saving model to tmp/EfficentNetB0_beans_model_e_02.h5
Epoch 3/300
33/33 [==============================] - 44s 1s/step - loss: 0.7894 - sparse_categorical_accuracy: 0.6756 - val_loss: 1.3099 - val_sparse_categorical_accuracy: 0.3308

Epoch 00003: saving model to tmp/EfficentNetB0_beans_model_e_03.h5
Epoch 4/300
33/33 [==============================] - 45s 1s/step - loss: 0.6316 - sparse_categorical_accuracy: 0.7251 - val_loss: 1.3615 - val_sparse_categorical_accuracy: 0.3308

Epoch 00004: saving model to tmp/EfficentNetB0_beans_model_e_04.h5
Epoch 5/300


33/33 [==============================] - 45s 1s/step - loss: 0.0968 - sparse_categorical_accuracy: 0.9626 - val_loss: 1.2128 - val_sparse_categorical_accuracy: 0.7744

Epoch 00034: saving model to tmp/EfficentNetB0_beans_model_e_34.h5
Epoch 35/300
33/33 [==============================] - 44s 1s/step - loss: 0.0235 - sparse_categorical_accuracy: 0.9910 - val_loss: 0.8073 - val_sparse_categorical_accuracy: 0.8045

Epoch 00035: saving model to tmp/EfficentNetB0_beans_model_e_35.h5
Epoch 36/300
33/33 [==============================] - 45s 1s/step - loss: 0.0797 - sparse_categorical_accuracy: 0.9765 - val_loss: 2.0149 - val_sparse_categorical_accuracy: 0.6241

Epoch 00036: saving model to tmp/EfficentNetB0_beans_model_e_36.h5
Epoch 37/300
33/33 [==============================] - 45s 1s/step - loss: 0.0793 - sparse_categorical_accuracy: 0.9757 - val_loss: 2.2425 - val_sparse_categorical_accuracy: 0.5714

Epoch 00037: saving model to tmp/EfficentNetB0_beans_model_e_37.h5
Epoch 38/300
33/33 [=

33/33 [==============================] - 45s 1s/step - loss: 0.0218 - sparse_categorical_accuracy: 0.9939 - val_loss: 0.7171 - val_sparse_categorical_accuracy: 0.7895

Epoch 00067: saving model to tmp/EfficentNetB0_beans_model_e_67.h5
Epoch 68/300
33/33 [==============================] - 44s 1s/step - loss: 0.0374 - sparse_categorical_accuracy: 0.9883 - val_loss: 0.6515 - val_sparse_categorical_accuracy: 0.8045

Epoch 00068: saving model to tmp/EfficentNetB0_beans_model_e_68.h5
Epoch 69/300
33/33 [==============================] - 44s 1s/step - loss: 0.0249 - sparse_categorical_accuracy: 0.9945 - val_loss: 0.9901 - val_sparse_categorical_accuracy: 0.7820

Epoch 00069: saving model to tmp/EfficentNetB0_beans_model_e_69.h5
Epoch 70/300
33/33 [==============================] - 45s 1s/step - loss: 0.1010 - sparse_categorical_accuracy: 0.9862 - val_loss: 1.1109 - val_sparse_categorical_accuracy: 0.7444

Epoch 00070: saving model to tmp/EfficentNetB0_beans_model_e_70.h5
Epoch 71/300
33/33 [=

33/33 [==============================] - 44s 1s/step - loss: 0.0457 - sparse_categorical_accuracy: 0.9877 - val_loss: 0.7509 - val_sparse_categorical_accuracy: 0.7744

Epoch 00100: saving model to tmp/EfficentNetB0_beans_model_e_100.h5
Epoch 101/300
33/33 [==============================] - 45s 1s/step - loss: 0.0232 - sparse_categorical_accuracy: 0.9924 - val_loss: 0.8328 - val_sparse_categorical_accuracy: 0.7895

Epoch 00101: saving model to tmp/EfficentNetB0_beans_model_e_101.h5
Epoch 102/300
33/33 [==============================] - 44s 1s/step - loss: 0.1206 - sparse_categorical_accuracy: 0.9565 - val_loss: 2.3333 - val_sparse_categorical_accuracy: 0.7218

Epoch 00102: saving model to tmp/EfficentNetB0_beans_model_e_102.h5
Epoch 103/300
33/33 [==============================] - 45s 1s/step - loss: 0.0467 - sparse_categorical_accuracy: 0.9841 - val_loss: 1.2764 - val_sparse_categorical_accuracy: 0.7444

Epoch 00103: saving model to tmp/EfficentNetB0_beans_model_e_103.h5
Epoch 104/300


Epoch 133/300
33/33 [==============================] - 45s 1s/step - loss: 0.0057 - sparse_categorical_accuracy: 0.9959 - val_loss: 0.5999 - val_sparse_categorical_accuracy: 0.8195

Epoch 00133: saving model to tmp/EfficentNetB0_beans_model_e_133.h5
Epoch 134/300
33/33 [==============================] - 44s 1s/step - loss: 0.0287 - sparse_categorical_accuracy: 0.9933 - val_loss: 0.6840 - val_sparse_categorical_accuracy: 0.8120

Epoch 00134: saving model to tmp/EfficentNetB0_beans_model_e_134.h5
Epoch 135/300
33/33 [==============================] - 45s 1s/step - loss: 0.0061 - sparse_categorical_accuracy: 0.9994 - val_loss: 0.7929 - val_sparse_categorical_accuracy: 0.8045

Epoch 00135: saving model to tmp/EfficentNetB0_beans_model_e_135.h5
Epoch 136/300
33/33 [==============================] - 45s 1s/step - loss: 0.0044 - sparse_categorical_accuracy: 0.9988 - val_loss: 0.9999 - val_sparse_categorical_accuracy: 0.7368

Epoch 00136: saving model to tmp/EfficentNetB0_beans_model_e_136.h5


Epoch 166/300
33/33 [==============================] - 44s 1s/step - loss: 0.0290 - sparse_categorical_accuracy: 0.9927 - val_loss: 1.3057 - val_sparse_categorical_accuracy: 0.7669

Epoch 00166: saving model to tmp/EfficentNetB0_beans_model_e_166.h5
Epoch 167/300
33/33 [==============================] - 45s 1s/step - loss: 0.0219 - sparse_categorical_accuracy: 0.9930 - val_loss: 0.7185 - val_sparse_categorical_accuracy: 0.7744

Epoch 00167: saving model to tmp/EfficentNetB0_beans_model_e_167.h5
Epoch 168/300
33/33 [==============================] - 45s 1s/step - loss: 0.0204 - sparse_categorical_accuracy: 0.9906 - val_loss: 0.8636 - val_sparse_categorical_accuracy: 0.7669

Epoch 00168: saving model to tmp/EfficentNetB0_beans_model_e_168.h5
Epoch 169/300
33/33 [==============================] - 44s 1s/step - loss: 0.0049 - sparse_categorical_accuracy: 0.9990 - val_loss: 0.7965 - val_sparse_categorical_accuracy: 0.8195

Epoch 00169: saving model to tmp/EfficentNetB0_beans_model_e_169.h5


Epoch 199/300
33/33 [==============================] - 44s 1s/step - loss: 0.0157 - sparse_categorical_accuracy: 0.9963 - val_loss: 1.1064 - val_sparse_categorical_accuracy: 0.7444

Epoch 00199: saving model to tmp/EfficentNetB0_beans_model_e_199.h5
Epoch 200/300
33/33 [==============================] - 45s 1s/step - loss: 0.0096 - sparse_categorical_accuracy: 0.9964 - val_loss: 1.0033 - val_sparse_categorical_accuracy: 0.7895

Epoch 00200: saving model to tmp/EfficentNetB0_beans_model_e_200.h5
Epoch 201/300
33/33 [==============================] - 45s 1s/step - loss: 0.0020 - sparse_categorical_accuracy: 0.9994 - val_loss: 1.0722 - val_sparse_categorical_accuracy: 0.7744

Epoch 00201: saving model to tmp/EfficentNetB0_beans_model_e_201.h5
Epoch 202/300
33/33 [==============================] - 45s 1s/step - loss: 0.0021 - sparse_categorical_accuracy: 0.9994 - val_loss: 1.0200 - val_sparse_categorical_accuracy: 0.7594

Epoch 00202: saving model to tmp/EfficentNetB0_beans_model_e_202.h5


Epoch 232/300
33/33 [==============================] - 45s 1s/step - loss: 0.0060 - sparse_categorical_accuracy: 0.9973 - val_loss: 1.1518 - val_sparse_categorical_accuracy: 0.7744

Epoch 00232: saving model to tmp/EfficentNetB0_beans_model_e_232.h5
Epoch 233/300
33/33 [==============================] - 45s 1s/step - loss: 0.0107 - sparse_categorical_accuracy: 0.9968 - val_loss: 1.3647 - val_sparse_categorical_accuracy: 0.6992

Epoch 00233: saving model to tmp/EfficentNetB0_beans_model_e_233.h5
Epoch 234/300
33/33 [==============================] - 45s 1s/step - loss: 0.0026 - sparse_categorical_accuracy: 1.0000 - val_loss: 1.0699 - val_sparse_categorical_accuracy: 0.7820

Epoch 00234: saving model to tmp/EfficentNetB0_beans_model_e_234.h5
Epoch 235/300
33/33 [==============================] - 45s 1s/step - loss: 0.0016 - sparse_categorical_accuracy: 0.9996 - val_loss: 0.9702 - val_sparse_categorical_accuracy: 0.8195

Epoch 00235: saving model to tmp/EfficentNetB0_beans_model_e_235.h5



Epoch 00264: saving model to tmp/EfficentNetB0_beans_model_e_264.h5
Epoch 265/300
33/33 [==============================] - 45s 1s/step - loss: 0.0193 - sparse_categorical_accuracy: 0.9943 - val_loss: 1.4264 - val_sparse_categorical_accuracy: 0.6992

Epoch 00265: saving model to tmp/EfficentNetB0_beans_model_e_265.h5
Epoch 266/300
33/33 [==============================] - 44s 1s/step - loss: 0.0065 - sparse_categorical_accuracy: 0.9959 - val_loss: 1.2413 - val_sparse_categorical_accuracy: 0.7368

Epoch 00266: saving model to tmp/EfficentNetB0_beans_model_e_266.h5
Epoch 267/300
33/33 [==============================] - 44s 1s/step - loss: 0.0025 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.9969 - val_sparse_categorical_accuracy: 0.7744

Epoch 00267: saving model to tmp/EfficentNetB0_beans_model_e_267.h5
Epoch 268/300
33/33 [==============================] - 45s 1s/step - loss: 0.0016 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.9056 - val_sparse_categorical_accuracy: 0.7594


33/33 [==============================] - 45s 1s/step - loss: 0.0036 - sparse_categorical_accuracy: 0.9990 - val_loss: 0.8668 - val_sparse_categorical_accuracy: 0.7820

Epoch 00297: saving model to tmp/EfficentNetB0_beans_model_e_297.h5
Epoch 298/300
33/33 [==============================] - 44s 1s/step - loss: 0.0032 - sparse_categorical_accuracy: 0.9989 - val_loss: 1.1285 - val_sparse_categorical_accuracy: 0.7594

Epoch 00298: saving model to tmp/EfficentNetB0_beans_model_e_298.h5
Epoch 299/300
33/33 [==============================] - 44s 1s/step - loss: 0.0051 - sparse_categorical_accuracy: 0.9983 - val_loss: 1.1546 - val_sparse_categorical_accuracy: 0.8195

Epoch 00299: saving model to tmp/EfficentNetB0_beans_model_e_299.h5
Epoch 300/300
33/33 [==============================] - 44s 1s/step - loss: 0.0070 - sparse_categorical_accuracy: 0.9964 - val_loss: 0.8381 - val_sparse_categorical_accuracy: 0.8195

Epoch 00300: saving model to tmp/EfficentNetB0_beans_model_e_300.h5
Saved to: Effi

#### EfficientNetB4

In [15]:
MODEL_PATH = 'EfficentNetB4_beans_model'

In [16]:
base_model = EfficientNetB4(include_top=False, weights=None, input_shape=INPUT_SHAPE)

model = wrap_model_for_beans(base_model=base_model, num_classes=NUM_CLASSES)

train_model(model=model, ds_train=ds_train, ds_validation=ds_validation, model_name=MODEL_PATH)

Epoch 1/300


ResourceExhaustedError:  OOM when allocating tensor with shape[10752] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node model_2/block3c_se_squeeze/Mean (defined at <ipython-input-3-66930933bfe0>:24) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_922807]

Function call stack:
train_function


## flowers dataset

https://www.tensorflow.org/datasets/catalog/oxford_flowers102

In [17]:
def random_crop(image):
    cropped_image = tf.image.random_crop(
        image, size=[256, 256, 3])

    return cropped_image

def random_jitter(image):
    # resizing to 286 x 286 x 3
    image = tf.image.resize(image, [286, 286],
                          method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

    # randomly cropping to 256 x 256 x 3
    image = random_crop(image)

    # random mirroring
    image = tf.image.random_flip_left_right(image)

    return image

def preprocess_flowers_train(image, label):
    image = random_jitter(image)
    return image, label

# -------------------------------

def preprocess_flowers(image, label):
    image = tf.image.resize(image, [256, 256],
                          method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    return image, label

### wrap model for flowers

In [18]:
def wrap_model_for_flowers(base_model, num_classes):
    inputs = base_model.inputs
    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.2)(x)
    outputs = layers.Dense(num_classes)(x)
    model = tf.keras.Model(inputs, outputs)
    return model

In [19]:
NUM_CLASSES = 102
INPUT_SHAPE = (256, 256, 3)
BATCH_SIZE = 32
RESIZE_DIMENSION = 256

In [20]:
def load_flowers_dataset():  
    (ds_train, ds_validation, ds_test), ds_info = tfds.load(name="tf_flowers", 
                                             with_info=True,
                                             split=['train[:70%]', 'train[70%:85%]', 'train[85%:]'],  #70/15/15 split
                                             as_supervised=True)

    ds_train = ds_train.map(normalize, 
                            num_parallel_calls=tf.data.experimental.AUTOTUNE)    
    ds_train = ds_train.map(preprocess_flowers)
    ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
    ds_train = ds_train.batch(BATCH_SIZE)
    ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)
    
    ds_validation = ds_validation.map(normalize, 
                          num_parallel_calls=tf.data.experimental.AUTOTUNE)
    ds_validation = ds_validation.map(preprocess_flowers)
    ds_validation = ds_validation.batch(BATCH_SIZE)
    ds_validation = ds_validation.prefetch(tf.data.experimental.AUTOTUNE)
    
    ds_test = ds_test.map(normalize, 
                          num_parallel_calls=tf.data.experimental.AUTOTUNE)
    ds_test = ds_test.map(preprocess_flowers)
    ds_test = ds_test.batch(BATCH_SIZE)
    ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)
    
    return ds_train, ds_validation, ds_test

In [21]:
ds_train, ds_validation, ds_test = load_flowers_dataset()

### mobilenetV2

In [22]:
MODEL_NAME = 'MobileNetV2_flowers_model'

In [23]:
base_model = MobileNetV2(include_top=False, weights=None, input_shape=INPUT_SHAPE)

model = wrap_model_for_flowers(base_model=base_model, num_classes=NUM_CLASSES)

train_model(model=model, ds_train=ds_train, ds_validation=ds_validation, model_name=MODEL_NAME)

Epoch 1/300
81/81 [==============================] - 34s 236ms/step - loss: 2.3661 - sparse_categorical_accuracy: 0.3575 - val_loss: 4.3391 - val_sparse_categorical_accuracy: 0.2668

Epoch 00001: saving model to tmp/MobileNetV2_flowers_model_e_01.h5
Epoch 2/300
81/81 [==============================] - 29s 224ms/step - loss: 1.1504 - sparse_categorical_accuracy: 0.5644 - val_loss: 4.0571 - val_sparse_categorical_accuracy: 0.2668

Epoch 00002: saving model to tmp/MobileNetV2_flowers_model_e_02.h5
Epoch 3/300
81/81 [==============================] - 29s 226ms/step - loss: 1.0400 - sparse_categorical_accuracy: 0.5973 - val_loss: 3.7580 - val_sparse_categorical_accuracy: 0.2668

Epoch 00003: saving model to tmp/MobileNetV2_flowers_model_e_03.h5
Epoch 4/300
81/81 [==============================] - 29s 224ms/step - loss: 0.9275 - sparse_categorical_accuracy: 0.6320 - val_loss: 3.4233 - val_sparse_categorical_accuracy: 0.2668

Epoch 00004: saving model to tmp/MobileNetV2_flowers_model_e_04.h5



Epoch 00033: saving model to tmp/MobileNetV2_flowers_model_e_33.h5
Epoch 34/300
81/81 [==============================] - 29s 224ms/step - loss: 0.1546 - sparse_categorical_accuracy: 0.9454 - val_loss: 8.5937 - val_sparse_categorical_accuracy: 0.2668

Epoch 00034: saving model to tmp/MobileNetV2_flowers_model_e_34.h5
Epoch 35/300
81/81 [==============================] - 29s 225ms/step - loss: 0.1054 - sparse_categorical_accuracy: 0.9648 - val_loss: 9.5211 - val_sparse_categorical_accuracy: 0.2668

Epoch 00035: saving model to tmp/MobileNetV2_flowers_model_e_35.h5
Epoch 36/300
81/81 [==============================] - 29s 225ms/step - loss: 0.0939 - sparse_categorical_accuracy: 0.9650 - val_loss: 9.7898 - val_sparse_categorical_accuracy: 0.2668

Epoch 00036: saving model to tmp/MobileNetV2_flowers_model_e_36.h5
Epoch 37/300
81/81 [==============================] - 30s 226ms/step - loss: 0.1384 - sparse_categorical_accuracy: 0.9522 - val_loss: 9.4237 - val_sparse_categorical_accuracy: 0.2

81/81 [==============================] - 30s 227ms/step - loss: 0.0523 - sparse_categorical_accuracy: 0.9817 - val_loss: 12.0353 - val_sparse_categorical_accuracy: 0.2668

Epoch 00066: saving model to tmp/MobileNetV2_flowers_model_e_66.h5
Epoch 67/300
81/81 [==============================] - 29s 225ms/step - loss: 0.0362 - sparse_categorical_accuracy: 0.9916 - val_loss: 12.5005 - val_sparse_categorical_accuracy: 0.2668

Epoch 00067: saving model to tmp/MobileNetV2_flowers_model_e_67.h5
Epoch 68/300
81/81 [==============================] - 29s 223ms/step - loss: 0.0365 - sparse_categorical_accuracy: 0.9860 - val_loss: 12.2187 - val_sparse_categorical_accuracy: 0.2668

Epoch 00068: saving model to tmp/MobileNetV2_flowers_model_e_68.h5
Epoch 69/300
81/81 [==============================] - 29s 225ms/step - loss: 0.1067 - sparse_categorical_accuracy: 0.9708 - val_loss: 11.8989 - val_sparse_categorical_accuracy: 0.2668

Epoch 00069: saving model to tmp/MobileNetV2_flowers_model_e_69.h5
Epoch


Epoch 00098: saving model to tmp/MobileNetV2_flowers_model_e_98.h5
Epoch 99/300
81/81 [==============================] - 29s 223ms/step - loss: 0.0104 - sparse_categorical_accuracy: 0.9978 - val_loss: 2.8886 - val_sparse_categorical_accuracy: 0.6189

Epoch 00099: saving model to tmp/MobileNetV2_flowers_model_e_99.h5
Epoch 100/300
81/81 [==============================] - 30s 225ms/step - loss: 0.0925 - sparse_categorical_accuracy: 0.9663 - val_loss: 5.1146 - val_sparse_categorical_accuracy: 0.5154

Epoch 00100: saving model to tmp/MobileNetV2_flowers_model_e_100.h5
Epoch 101/300
81/81 [==============================] - 29s 224ms/step - loss: 0.0609 - sparse_categorical_accuracy: 0.9788 - val_loss: 4.5641 - val_sparse_categorical_accuracy: 0.5263

Epoch 00101: saving model to tmp/MobileNetV2_flowers_model_e_101.h5
Epoch 102/300
81/81 [==============================] - 30s 224ms/step - loss: 0.0671 - sparse_categorical_accuracy: 0.9788 - val_loss: 4.6656 - val_sparse_categorical_accuracy

81/81 [==============================] - 29s 223ms/step - loss: 0.0174 - sparse_categorical_accuracy: 0.9940 - val_loss: 4.6065 - val_sparse_categorical_accuracy: 0.5753

Epoch 00131: saving model to tmp/MobileNetV2_flowers_model_e_131.h5
Epoch 132/300
81/81 [==============================] - 29s 223ms/step - loss: 0.0159 - sparse_categorical_accuracy: 0.9934 - val_loss: 4.9787 - val_sparse_categorical_accuracy: 0.5681

Epoch 00132: saving model to tmp/MobileNetV2_flowers_model_e_132.h5
Epoch 133/300
81/81 [==============================] - 29s 224ms/step - loss: 0.0119 - sparse_categorical_accuracy: 0.9967 - val_loss: 3.8844 - val_sparse_categorical_accuracy: 0.6334

Epoch 00133: saving model to tmp/MobileNetV2_flowers_model_e_133.h5
Epoch 134/300
81/81 [==============================] - 29s 224ms/step - loss: 0.0120 - sparse_categorical_accuracy: 0.9961 - val_loss: 4.9554 - val_sparse_categorical_accuracy: 0.5064

Epoch 00134: saving model to tmp/MobileNetV2_flowers_model_e_134.h5
Ep

81/81 [==============================] - 30s 223ms/step - loss: 0.0492 - sparse_categorical_accuracy: 0.9837 - val_loss: 6.3972 - val_sparse_categorical_accuracy: 0.5227

Epoch 00163: saving model to tmp/MobileNetV2_flowers_model_e_163.h5
Epoch 164/300
81/81 [==============================] - 30s 227ms/step - loss: 0.0355 - sparse_categorical_accuracy: 0.9839 - val_loss: 7.1084 - val_sparse_categorical_accuracy: 0.4719

Epoch 00164: saving model to tmp/MobileNetV2_flowers_model_e_164.h5
Epoch 165/300
81/81 [==============================] - 29s 223ms/step - loss: 0.1043 - sparse_categorical_accuracy: 0.9682 - val_loss: 6.9590 - val_sparse_categorical_accuracy: 0.5263

Epoch 00165: saving model to tmp/MobileNetV2_flowers_model_e_165.h5
Epoch 166/300
81/81 [==============================] - 29s 225ms/step - loss: 0.0242 - sparse_categorical_accuracy: 0.9918 - val_loss: 5.3232 - val_sparse_categorical_accuracy: 0.5699

Epoch 00166: saving model to tmp/MobileNetV2_flowers_model_e_166.h5
Ep

81/81 [==============================] - 29s 223ms/step - loss: 0.0154 - sparse_categorical_accuracy: 0.9938 - val_loss: 3.1885 - val_sparse_categorical_accuracy: 0.6171

Epoch 00195: saving model to tmp/MobileNetV2_flowers_model_e_195.h5
Epoch 196/300
81/81 [==============================] - 29s 225ms/step - loss: 0.0651 - sparse_categorical_accuracy: 0.9792 - val_loss: 6.2434 - val_sparse_categorical_accuracy: 0.4737

Epoch 00196: saving model to tmp/MobileNetV2_flowers_model_e_196.h5
Epoch 197/300
81/81 [==============================] - 30s 225ms/step - loss: 0.0642 - sparse_categorical_accuracy: 0.9813 - val_loss: 8.6860 - val_sparse_categorical_accuracy: 0.4483

Epoch 00197: saving model to tmp/MobileNetV2_flowers_model_e_197.h5
Epoch 198/300
81/81 [==============================] - 28s 226ms/step - loss: 0.0499 - sparse_categorical_accuracy: 0.9826 - val_loss: 6.9607 - val_sparse_categorical_accuracy: 0.4229

Epoch 00198: saving model to tmp/MobileNetV2_flowers_model_e_198.h5
Ep

81/81 [==============================] - 29s 225ms/step - loss: 0.0537 - sparse_categorical_accuracy: 0.9814 - val_loss: 8.8451 - val_sparse_categorical_accuracy: 0.4864

Epoch 00227: saving model to tmp/MobileNetV2_flowers_model_e_227.h5
Epoch 228/300
81/81 [==============================] - 29s 225ms/step - loss: 0.0183 - sparse_categorical_accuracy: 0.9936 - val_loss: 5.4286 - val_sparse_categorical_accuracy: 0.5808

Epoch 00228: saving model to tmp/MobileNetV2_flowers_model_e_228.h5
Epoch 229/300
81/81 [==============================] - 30s 224ms/step - loss: 0.0083 - sparse_categorical_accuracy: 0.9975 - val_loss: 7.2273 - val_sparse_categorical_accuracy: 0.5082

Epoch 00229: saving model to tmp/MobileNetV2_flowers_model_e_229.h5
Epoch 230/300
81/81 [==============================] - 29s 223ms/step - loss: 0.0180 - sparse_categorical_accuracy: 0.9937 - val_loss: 6.6808 - val_sparse_categorical_accuracy: 0.4955

Epoch 00230: saving model to tmp/MobileNetV2_flowers_model_e_230.h5
Ep

81/81 [==============================] - 29s 226ms/step - loss: 0.0515 - sparse_categorical_accuracy: 0.9785 - val_loss: 6.8501 - val_sparse_categorical_accuracy: 0.4682

Epoch 00259: saving model to tmp/MobileNetV2_flowers_model_e_259.h5
Epoch 260/300
81/81 [==============================] - 30s 225ms/step - loss: 0.0393 - sparse_categorical_accuracy: 0.9855 - val_loss: 5.5320 - val_sparse_categorical_accuracy: 0.5771

Epoch 00260: saving model to tmp/MobileNetV2_flowers_model_e_260.h5
Epoch 261/300
81/81 [==============================] - 28s 221ms/step - loss: 0.0248 - sparse_categorical_accuracy: 0.9921 - val_loss: 3.4312 - val_sparse_categorical_accuracy: 0.6007

Epoch 00261: saving model to tmp/MobileNetV2_flowers_model_e_261.h5
Epoch 262/300
81/81 [==============================] - 30s 222ms/step - loss: 0.0397 - sparse_categorical_accuracy: 0.9878 - val_loss: 11.6860 - val_sparse_categorical_accuracy: 0.4065

Epoch 00262: saving model to tmp/MobileNetV2_flowers_model_e_262.h5
E

81/81 [==============================] - 30s 225ms/step - loss: 0.0154 - sparse_categorical_accuracy: 0.9960 - val_loss: 4.6347 - val_sparse_categorical_accuracy: 0.5408

Epoch 00291: saving model to tmp/MobileNetV2_flowers_model_e_291.h5
Epoch 292/300
81/81 [==============================] - 29s 224ms/step - loss: 0.0108 - sparse_categorical_accuracy: 0.9968 - val_loss: 4.6621 - val_sparse_categorical_accuracy: 0.5445

Epoch 00292: saving model to tmp/MobileNetV2_flowers_model_e_292.h5
Epoch 293/300
81/81 [==============================] - 29s 224ms/step - loss: 0.0061 - sparse_categorical_accuracy: 0.9984 - val_loss: 3.5522 - val_sparse_categorical_accuracy: 0.6044

Epoch 00293: saving model to tmp/MobileNetV2_flowers_model_e_293.h5
Epoch 294/300
81/81 [==============================] - 29s 223ms/step - loss: 0.0106 - sparse_categorical_accuracy: 0.9973 - val_loss: 3.1229 - val_sparse_categorical_accuracy: 0.6552

Epoch 00294: saving model to tmp/MobileNetV2_flowers_model_e_294.h5
Ep

### EfficientNets - B0, B4

#### EfficientNetB0

In [24]:
MODEL_NAME = 'EfficentNetB0_flowers_model'

In [ ]:
base_model = EfficientNetB0(include_top=False, weights=None, input_shape=INPUT_SHAPE)

model = wrap_model_for_flowers(base_model=base_model, num_classes=NUM_CLASSES)

train_model(model=model, ds_train=ds_train, ds_validation=ds_validation, model_name=MODEL_NAME)

Epoch 1/300
81/81 [==============================] - 43s 335ms/step - loss: 3.0777 - sparse_categorical_accuracy: 0.2806 - val_loss: 2.6566 - val_sparse_categorical_accuracy: 0.2668

Epoch 00001: saving model to tmp/EfficentNetB0_flowers_model_e_01.h5
Epoch 2/300
81/81 [==============================] - 36s 319ms/step - loss: 1.3022 - sparse_categorical_accuracy: 0.4645 - val_loss: 2.1797 - val_sparse_categorical_accuracy: 0.2668

Epoch 00002: saving model to tmp/EfficentNetB0_flowers_model_e_02.h5
Epoch 3/300
81/81 [==============================] - 37s 319ms/step - loss: 1.0181 - sparse_categorical_accuracy: 0.5835 - val_loss: 2.1402 - val_sparse_categorical_accuracy: 0.2668

Epoch 00003: saving model to tmp/EfficentNetB0_flowers_model_e_03.h5
Epoch 4/300
81/81 [==============================] - 37s 322ms/step - loss: 0.9478 - sparse_categorical_accuracy: 0.6399 - val_loss: 2.1931 - val_sparse_categorical_accuracy: 0.2668

Epoch 00004: saving model to tmp/EfficentNetB0_flowers_model_

81/81 [==============================] - 37s 319ms/step - loss: 0.0970 - sparse_categorical_accuracy: 0.9685 - val_loss: 1.6560 - val_sparse_categorical_accuracy: 0.6407

Epoch 00033: saving model to tmp/EfficentNetB0_flowers_model_e_33.h5
Epoch 34/300
81/81 [==============================] - 37s 320ms/step - loss: 0.1074 - sparse_categorical_accuracy: 0.9642 - val_loss: 1.7653 - val_sparse_categorical_accuracy: 0.6171

Epoch 00034: saving model to tmp/EfficentNetB0_flowers_model_e_34.h5
Epoch 35/300
81/81 [==============================] - 37s 322ms/step - loss: 0.0919 - sparse_categorical_accuracy: 0.9691 - val_loss: 1.6630 - val_sparse_categorical_accuracy: 0.6243

Epoch 00035: saving model to tmp/EfficentNetB0_flowers_model_e_35.h5
Epoch 36/300
81/81 [==============================] - 37s 321ms/step - loss: 0.1295 - sparse_categorical_accuracy: 0.9525 - val_loss: 2.5273 - val_sparse_categorical_accuracy: 0.5935

Epoch 00036: saving model to tmp/EfficentNetB0_flowers_model_e_36.h5
E

81/81 [==============================] - 37s 319ms/step - loss: 0.0541 - sparse_categorical_accuracy: 0.9849 - val_loss: 1.6245 - val_sparse_categorical_accuracy: 0.6642

Epoch 00065: saving model to tmp/EfficentNetB0_flowers_model_e_65.h5
Epoch 66/300
81/81 [==============================] - 37s 321ms/step - loss: 0.0328 - sparse_categorical_accuracy: 0.9834 - val_loss: 1.4426 - val_sparse_categorical_accuracy: 0.6733

Epoch 00066: saving model to tmp/EfficentNetB0_flowers_model_e_66.h5
Epoch 67/300
81/81 [==============================] - 37s 320ms/step - loss: 0.0817 - sparse_categorical_accuracy: 0.9792 - val_loss: 1.8542 - val_sparse_categorical_accuracy: 0.6515

Epoch 00067: saving model to tmp/EfficentNetB0_flowers_model_e_67.h5
Epoch 68/300
81/81 [==============================] - 37s 319ms/step - loss: 0.0709 - sparse_categorical_accuracy: 0.9769 - val_loss: 1.7268 - val_sparse_categorical_accuracy: 0.6624

Epoch 00068: saving model to tmp/EfficentNetB0_flowers_model_e_68.h5
E

81/81 [==============================] - 37s 320ms/step - loss: 0.0453 - sparse_categorical_accuracy: 0.9856 - val_loss: 1.8084 - val_sparse_categorical_accuracy: 0.6733

Epoch 00097: saving model to tmp/EfficentNetB0_flowers_model_e_97.h5
Epoch 98/300
81/81 [==============================] - 37s 322ms/step - loss: 0.0297 - sparse_categorical_accuracy: 0.9890 - val_loss: 1.7663 - val_sparse_categorical_accuracy: 0.6697

Epoch 00098: saving model to tmp/EfficentNetB0_flowers_model_e_98.h5
Epoch 99/300
81/81 [==============================] - 37s 321ms/step - loss: 0.0298 - sparse_categorical_accuracy: 0.9919 - val_loss: 1.6043 - val_sparse_categorical_accuracy: 0.6878

Epoch 00099: saving model to tmp/EfficentNetB0_flowers_model_e_99.h5
Epoch 100/300
81/81 [==============================] - 37s 320ms/step - loss: 0.0591 - sparse_categorical_accuracy: 0.9792 - val_loss: 2.5218 - val_sparse_categorical_accuracy: 0.6479

Epoch 00100: saving model to tmp/EfficentNetB0_flowers_model_e_100.h5

81/81 [==============================] - 37s 321ms/step - loss: 0.0320 - sparse_categorical_accuracy: 0.9891 - val_loss: 2.2201 - val_sparse_categorical_accuracy: 0.5898

Epoch 00129: saving model to tmp/EfficentNetB0_flowers_model_e_129.h5
Epoch 130/300
81/81 [==============================] - 37s 321ms/step - loss: 0.0270 - sparse_categorical_accuracy: 0.9898 - val_loss: 2.3235 - val_sparse_categorical_accuracy: 0.6279

Epoch 00130: saving model to tmp/EfficentNetB0_flowers_model_e_130.h5
Epoch 131/300
81/81 [==============================] - 36s 321ms/step - loss: 0.0225 - sparse_categorical_accuracy: 0.9924 - val_loss: 1.6633 - val_sparse_categorical_accuracy: 0.6770

Epoch 00131: saving model to tmp/EfficentNetB0_flowers_model_e_131.h5
Epoch 132/300
81/81 [==============================] - 36s 318ms/step - loss: 0.0247 - sparse_categorical_accuracy: 0.9917 - val_loss: 2.4038 - val_sparse_categorical_accuracy: 0.6025

Epoch 00132: saving model to tmp/EfficentNetB0_flowers_model_e_1

81/81 [==============================] - 37s 321ms/step - loss: 0.0162 - sparse_categorical_accuracy: 0.9938 - val_loss: 1.7747 - val_sparse_categorical_accuracy: 0.6624

Epoch 00161: saving model to tmp/EfficentNetB0_flowers_model_e_161.h5
Epoch 162/300
81/81 [==============================] - 36s 322ms/step - loss: 0.0186 - sparse_categorical_accuracy: 0.9934 - val_loss: 1.7907 - val_sparse_categorical_accuracy: 0.6806

Epoch 00162: saving model to tmp/EfficentNetB0_flowers_model_e_162.h5
Epoch 163/300
81/81 [==============================] - 37s 318ms/step - loss: 0.0121 - sparse_categorical_accuracy: 0.9971 - val_loss: 1.6565 - val_sparse_categorical_accuracy: 0.6969

Epoch 00163: saving model to tmp/EfficentNetB0_flowers_model_e_163.h5
Epoch 164/300
81/81 [==============================] - 37s 319ms/step - loss: 0.0174 - sparse_categorical_accuracy: 0.9946 - val_loss: 1.6309 - val_sparse_categorical_accuracy: 0.6969

Epoch 00164: saving model to tmp/EfficentNetB0_flowers_model_e_1

81/81 [==============================] - 37s 319ms/step - loss: 0.0110 - sparse_categorical_accuracy: 0.9953 - val_loss: 2.0727 - val_sparse_categorical_accuracy: 0.6624

Epoch 00193: saving model to tmp/EfficentNetB0_flowers_model_e_193.h5
Epoch 194/300
81/81 [==============================] - 37s 319ms/step - loss: 0.0358 - sparse_categorical_accuracy: 0.9902 - val_loss: 2.6305 - val_sparse_categorical_accuracy: 0.5971

Epoch 00194: saving model to tmp/EfficentNetB0_flowers_model_e_194.h5
Epoch 195/300
81/81 [==============================] - 37s 319ms/step - loss: 0.0408 - sparse_categorical_accuracy: 0.9845 - val_loss: 2.1359 - val_sparse_categorical_accuracy: 0.6497

Epoch 00195: saving model to tmp/EfficentNetB0_flowers_model_e_195.h5
Epoch 196/300
81/81 [==============================] - 37s 323ms/step - loss: 0.0306 - sparse_categorical_accuracy: 0.9903 - val_loss: 1.7453 - val_sparse_categorical_accuracy: 0.6897

Epoch 00196: saving model to tmp/EfficentNetB0_flowers_model_e_1

81/81 [==============================] - 36s 319ms/step - loss: 0.0106 - sparse_categorical_accuracy: 0.9944 - val_loss: 1.8471 - val_sparse_categorical_accuracy: 0.6661

Epoch 00225: saving model to tmp/EfficentNetB0_flowers_model_e_225.h5
Epoch 226/300
81/81 [==============================] - 37s 321ms/step - loss: 0.0321 - sparse_categorical_accuracy: 0.9913 - val_loss: 2.2367 - val_sparse_categorical_accuracy: 0.6552

Epoch 00226: saving model to tmp/EfficentNetB0_flowers_model_e_226.h5
Epoch 227/300
81/81 [==============================] - 38s 320ms/step - loss: 0.0176 - sparse_categorical_accuracy: 0.9957 - val_loss: 1.7300 - val_sparse_categorical_accuracy: 0.7223

Epoch 00227: saving model to tmp/EfficentNetB0_flowers_model_e_227.h5
Epoch 228/300
81/81 [==============================] - 37s 319ms/step - loss: 0.0171 - sparse_categorical_accuracy: 0.9934 - val_loss: 2.0916 - val_sparse_categorical_accuracy: 0.6679

Epoch 00228: saving model to tmp/EfficentNetB0_flowers_model_e_2

#### EfficientNetB4

In [ ]:
MODEL_PATH = './flowers_models/EfficentNetB4_flowers_model.h5'

In [ ]:
base_model = EfficientNetB4(include_top=False, weights=None, input_shape=INPUT_SHAPE)

model = wrap_model_for_beans(base_model=base_model, num_classes=NUM_CLASSES)

train_model(model=model, ds_train=ds_train, ds_validation=ds_validation, model_path=MODEL_PATH)